<a href="https://colab.research.google.com/github/juanflorezVe/COMP9058_lab1/blob/master/r00184264_juanflorez_assg1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
# Copying vervatim the code from the assignment
import tensorflow as tf
print(tf.__version__)
experiment = tf.Variable([1,2,3])
print(tf.divide(tf.reduce_sum(experiment),experiment.shape[0]))

2.4.1
tf.Tensor(2.0, shape=(), dtype=float64)


In [3]:
import tensorflow as tf
import numpy as np

from keras.utils import np_utils

fashion_mnist = tf.keras.datasets.fashion_mnist

# load the training and test data    
(tr_x, tr_y), (te_x, te_y) = fashion_mnist.load_data()

# reshape the feature data
tr_x = tr_x.reshape(tr_x.shape[0], 784)
te_x = te_x.reshape(te_x.shape[0], 784)

# noramlise feature data
tr_x = tr_x / 255.0
te_x = te_x / 255.0

print( "Shape of training features ", tr_x.shape)
print( "Shape of test features ", te_x.shape)


# one hot encode the training labels and get the transpose
tr_y = np_utils.to_categorical(tr_y,10)
tr_y = tr_y.T
print ("Shape of training labels ", tr_y.shape)

# one hot encode the test labels and get the transpose
te_y = np_utils.to_categorical(te_y,10)
te_y = te_y.T
print ("Shape of testing labels ", te_y.shape)

4423680/4422102 [==============================] - 0s 0us/step
Shape of training features  (60000, 784)
Shape of test features  (10000, 784)
Shape of training labels  (10, 60000)
Shape of testing labels  (10, 10000)


In [201]:
#Let's insert 784 parameters to each neuron
# A = W.T * X + b
# layer_pass will take the x matrix, pass it to the l layer applying the w weights and adding the b bias
#NeuralNetworksW5 slide 11
# m = 60000
# n = 784
# p = 200
def relu_layer_pass(w, x, b):
  t = tf.matmul(w,x)
  l = tf.add(t,b)
  h = tf.nn.relu(l)
  return h

def softmax_layer_pass(w, x, b):
  t = tf.matmul(w,x)
  l = tf.add(t,b)
  h = tf.nn.softmax(l)
  return h

#TODO: X is the matrix with the training instances,
#but I need a function that given an Error, E, and a
# vector with inputs, gives me the d_lambda and db

def loss_function(E, x=None):
  db = tf.divide(tf.reduce_sum(E),E.shape[0])
  print("shape of x {}, shape of E transposed {}".
        format(x.shape, tf.transpose(E).shape))
  temp = tf.matmul(x, tf.transpose(E))
  d_lambda = tf.divide(temp, E.shape[0])
  return db, d_lambda

In [202]:
def forward_pass(w1, w2, b1, b2, x):
  #pass the first layer
  h1 = relu_layer_pass(w1,x,b1)
  #pass the second layer
  h2 = softmax_layer_pass(w2,h1,b2)
  return h2
  #print(H1.shape)
  #print(H2.shape)

In [203]:
# from lecture (VectorizedLogistiRegression, page 50) March 9th.... 
# We have a set of trainning of 60000 rows (instances)
# and 748 columns ( Features)
# Our X vector has 784 rows (as fetarues) and 60000 columns (instances)
X = tf.transpose(tr_x)
X = tf.cast(X, tf.float32)
print("X shape is {}".format( X.shape))
# Let's create a vector W1 with the number of weights (784) for each neuron in one row
# so, it means 200 rows and 784 columns ( Features) for layer 1
W1 = tf.Variable(tf.random.normal([ 200,784], mean=0.0, stddev=0.05))
#The bias for the first layer of 200 neurons will be a vecoter of 200 rows (1 per neuron)
# and 1 column
B1 = tf.Variable(tf.random.uniform(shape=[200,1],dtype=tf.float32))


print ("W1 shape is{}".format(W1.shape))
print ("B1 shape is{}".format(B1.shape))

X shape is (784, 60000)
W1 shape is(200, 784)
B1 shape is(200, 1)


In [204]:
#The second layer:
# W2 has 10 rows (as it has 10 possible classes)
W2 = tf.Variable(tf.random.uniform(shape=[10,200], dtype=tf.float32, minval=-0.1, maxval=0.1))
B2 = tf.Variable(tf.random.uniform(shape=[10,1], dtype=tf.float32))
print ("W2 shape is{}".format(W2.shape))
print ("B2 shape is{}".format(B2.shape))

W2 shape is(10, 200)
B2 shape is(10, 1)


Question 1, section a. Question1_1_1


In [205]:
learning_rate=0.01
iterations=1
adam_optimizer=tf.keras.optimizers.Adam()


In [214]:
# Rgression loop
for i in range(iterations):
  H = forward_pass(W1,W2,B1,B2,X)
  E = tf.subtract(H,tr_y)
  print("E is {}".format(E))
  print("W1 is {}".format(W1))
  print("H is {}".format(H))
  print("X is {}".format(X))
  db1, dl1 = loss_function(E, X)
  B1 = tf.subtract(B1, tf.scalar_mul(learning_rate, db1))
  secondpart = tf.scalar_mul(learning_rate, dl1)
  W1 = tf.subtract(W1,secondpart)
print("Done") 
    

E is [[ 2.18038222e-05 -9.99991119e-01 -9.99981642e-01 ...  2.52526279e-05
  -9.99986649e-01  1.59846986e-05]
 [ 1.53523506e-05  2.13291478e-05  2.01580879e-05 ...  1.94460954e-05
   1.58945859e-05  1.89139937e-05]
 [ 2.55389641e-05  1.71687934e-05  1.39249441e-05 ...  3.44154323e-05
   1.23632235e-05  1.26235245e-05]
 ...
 [ 2.10332837e-05  1.15374787e-05  1.20182422e-05 ...  1.97294539e-05
   1.60928867e-05  1.38193564e-05]
 [ 1.68148854e-05  1.54480895e-05  1.58720268e-05 ...  1.35970549e-05
   1.66821701e-05  1.30497301e-05]
 [-9.99975085e-01  1.07741898e-05  1.69434879e-05 ...  1.87824407e-05
   1.40612938e-05  2.07337162e-05]]
W1 is <tf.Variable 'Variable:0' shape=(200, 784) dtype=float32, numpy=
array([[-0.10522868, -0.01616263, -0.11177293, ...,  0.08140687,
         0.04514502,  0.05821004],
       [-0.00986829, -0.03909342, -0.00266622, ..., -0.00720748,
        -0.05664061,  0.05862942],
       [ 0.00145401,  0.01184191, -0.03829617, ...,  0.00057218,
         0.01849945,  0

InvalidArgumentError: ignored

In [213]:
#Unit tests
xu = tf.Variable([[0.3,0.2,0.1],
                  [0.7,0.6,0.5],
                  [0.2,0.2,0.2]])
big = tf.Variable([[6.0,8.0,10.0]], shape = [1,3])
small = tf.Variable([5.0,7.0,9.0])
E = tf.math.subtract(big, small)
minus_E = tf.math.subtract(small, big)
print(loss_function(E, xu)) # should be xu/3
print(loss_function(minus_E, xu)) # should be -xu/3

shape of x (3, 3), shape of E transposed (3, 1)
(<tf.Tensor: shape=(), dtype=float32, numpy=3.0>, <tf.Tensor: shape=(3, 1), dtype=float32, numpy=
array([[0.6],
       [1.8],
       [0.6]], dtype=float32)>)
shape of x (3, 3), shape of E transposed (3, 1)
(<tf.Tensor: shape=(), dtype=float32, numpy=-3.0>, <tf.Tensor: shape=(3, 1), dtype=float32, numpy=
array([[-0.6],
       [-1.8],
       [-0.6]], dtype=float32)>)


In [193]:
scalar = tf.Variable(3.0)
vector = tf.Variable([3,6,9], dtype=tf.float32)
prod = tf.scalar_mul(scalar, vector)
prod

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([ 9., 18., 27.], dtype=float32)>